# 02 - 데이터프레임 결합


## 1.환경준비

* 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

* 데이터 가져오기

In [2]:
sales = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_sales_simple.csv")

sales1 = sales.loc[sales['Store_ID']==1]
sales2 = sales.loc[sales['Store_ID']==2]
sales5 = sales.loc[sales['Store_ID']==5]
sales11 = sales.loc[sales['Store_ID']==11]

products = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_product_master.csv")
stores = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_store_master.csv")

In [3]:
sales1.head()

,Order_ID,Date,Store_ID,Qty,Product_ID
0,3,2013-01-01,1,0.0,3
1,5,2013-01-01,1,0.0,5
2,7,2013-01-01,1,0.0,7
3,8,2013-01-01,1,0.0,8
4,10,2013-01-01,1,0.0,10


In [4]:
products.head()

,Product_ID,Product_Code,SubCategory,Category,LeadTime,Price
0,20,HG001,Gardening Tools,Household Goods,2,50
1,27,HH001,Home Appliances,Household Goods,2,150
2,16,HK001,Kitchen,Household Goods,2,23
3,15,HK002,Kitchen,Household Goods,2,41
4,32,GS001,Seafood,Grocery,3,34


In [5]:
stores.head()

,Store_ID,City,State,Store_Type
0,1,Saint Paul,Minnesota,4
1,2,Saint Paul,Minnesota,4
2,3,Saint Paul,Minnesota,4
3,4,Saint Paul,Minnesota,4
4,5,Oklahoma City,Oklahoma,4


## 2.결합1 : 붙이기 concat()

* 데이터를 생성해 봅시다.

In [9]:
df1 = pd.DataFrame({'A':[10,25], 'B':[15,30]})
df2 = pd.DataFrame({'A':[20,30, 50], 'C':[35,30, 40]})
df2.drop([1],axis= 0, inplace = True)

display(df1)
display(df2)

,A,B
0,10,15
1,25,30


,A,C
0,20,35
2,50,40


#### (1) 세로로 합치기 : axis = 0

* join = 'inner'

In [14]:
display(df1)
display(df2)
pd.concat([df1, df2], axis = 0, join = 'inner')

,A,B
0,10,15
1,25,30


,A,C
0,20,35
2,50,40


,A
0,10
1,25
0,20
2,50


* join = 'outer'

In [11]:
pd.concat([df1, df2], axis = 0, join = 'outer')

,A,B,C
0,10,15.0,NaN
1,25,30.0,NaN
0,20,NaN,35.0
2,50,NaN,40.0


#### (2) 가로로 합치기 : axis = 1

* join = 'inner'

In [13]:
display(df1)
display(df2)
pd.concat([df1, df2], axis = 1, join = 'inner')

,A,B
0,10,15
1,25,30


,A,C
0,20,35
2,50,40


,A,B,A,C
0,10,15,20,35


* join = 'outer'

In [15]:
pd.concat([df1, df2], axis = 1, join = 'outer')

,A,B,A,C
0,10.0,15.0,20.0,35.0
1,25.0,30.0,NaN,NaN
2,NaN,NaN,50.0,40.0


<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] sales1,2,5,11 데이터프레임을 하나로 합칩시다. 어떻게 합치는게 좋을까요?

In [18]:
display(sales1.head(3))
display(sales2.head(3))
display(sales5.head(3))
display(sales11.head(3))

,Order_ID,Date,Store_ID,Qty,Product_ID
0,3,2013-01-01,1,0.0,3
1,5,2013-01-01,1,0.0,5
2,7,2013-01-01,1,0.0,7


,Order_ID,Date,Store_ID,Qty,Product_ID
165,366,2013-01-01,2,0.0,3
166,368,2013-01-01,2,0.0,5
167,370,2013-01-01,2,0.0,7


,Order_ID,Date,Store_ID,Qty,Product_ID
660,1455,2013-01-01,5,0.0,3
661,1457,2013-01-01,5,0.0,5
662,1459,2013-01-01,5,0.0,7


,Order_ID,Date,Store_ID,Qty,Product_ID
30,69,2013-01-01,11,0.0,3
31,71,2013-01-01,11,0.0,5
32,73,2013-01-01,11,0.0,7


In [26]:
sales0 = pd.concat([sales1,sales2,sales5,sales11],join="inner",axis=0)
sales0
#concat은 실전에서 비슷한 데이터를 합칠 때 많이 쓰임
# 위아래로 합칠 때 특히

,Order_ID,Date,Store_ID,Qty,Product_ID
0,3,2013-01-01,1,0.000,3
1,5,2013-01-01,1,0.000,5
2,7,2013-01-01,1,0.000,7
3,8,2013-01-01,1,0.000,8
4,10,2013-01-01,1,0.000,10
...,...,...,...,...,...
24340,53548,2013-01-31,11,10.000,22
24341,53550,2013-01-31,11,641.507,24
24342,53553,2013-01-31,11,0.000,27
24343,53555,2013-01-31,11,27.000,29


[문2] (조금 어려운 문제!)  
일별, 매장별 판매량 합계를 구해 봅시다.  
groupby, pd.concat 을 이용하여 다음과 같은 결과를 얻도록 코드를 작성해봅시다.

* 힌트
    * 1) sales1, 2, 5,11 로 부터 각각 일별 판매량 합계 집계하기
        * 집계시 .agg() 를 이용하여 열이름 변경하거나, 집계후 열이름 변경하기
    * 2) pd.concat 으로 가로로 합치기



## 3.결합 ① : 조인 join(merge)

* 판다스에서 join은 굉장히 간단합니다.
* 자동으로 key를 잡아줍니다
* default로 inner join을 합니다.

In [28]:
df1 = pd.DataFrame({'A':[1,2], 'B':[15,30], 'C':[20, 25]})
df2 = pd.DataFrame({'A':[2,3], 'D':[20, 35]})

display(df1)
display(df2)

,A,B,C
0,1,15,20
1,2,30,25


,A,D
0,2,20
1,3,35


### (1) inner merge

In [29]:
pd.merge(df1, df2, how = 'inner', on = 'A')

,A,B,C,D
0,2,30,25,20


In [ ]:
pd.merge(df1, df2, how = 'inner')

### (2) left merge

In [ ]:
pd.merge(df1, df2, how = 'left')

### (3) right merge

In [ ]:
pd.merge(df1, df2, how = 'right')

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] 일별, 매장별 판매량 합계를 구해 봅시다.  
groupby, pd.merge 을 이용하여 다음과 같은 결과를 얻도록 코드를 작성해봅시다.

* 힌트
    * 1) sales1, 2, 5,11 로 부터 각각 일별 판매량 합계 집계하기
        * 집계시 .agg() 를 이용하여 열이름 변경하거나, 집계후 열이름 변경하기
    * 2) pd.merge 으로 결합하기



[문2] 매장1에 대해서, 일별, 카테고리별 판매량 합계를 구해 봅시다. 



[문3] 도시별, 총판매량 합계를 구하시오.  



### (4) pivot
* pivot : 결합은 아니지만, 집계 후 데이터프레임 구조를 변형해서 조회하는데 종종 사용 됩니다.

* 단계
    * 1) groupby
    * 2) pivot

In [75]:
# 1) 매장1의 일별 카테고리별 판매량을 집계

temp = pd.merge(sales1, products)
temp2 = temp.groupby(['Date', 'Category'], as_index = False)['Qty'].sum()
temp2

,Date,Category,Qty
0,2013-01-01,Drink,0.000
1,2013-01-01,Food,0.000
2,2013-01-01,Grocery,0.000
3,2013-01-01,Household Goods,0.000
4,2013-01-02,Drink,1158.000
...,...,...,...
119,2013-01-30,Household Goods,932.000
120,2013-01-31,Drink,971.000
121,2013-01-31,Food,751.766
122,2013-01-31,Grocery,1907.797


In [ ]:
# 2) pivot
temp3 = temp2.pivot( 'Category', 'Date' ,'Qty')
temp3

In [ ]:
# sns.heatmap으로 시각화 할 수도 있습니다!

plt.figure(figsize = (20, 6))
sns.heatmap(temp3)
plt.show()

## 4.복습문제

In [ ]:
# 라이브러리 불러오기
import pandas as pd

In [3]:
sales = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_sales_simple.csv")
products = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_product_master.csv")
stores = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_store_master.csv")

In [32]:
sales.head(3)

,Order_ID,Date,Store_ID,Qty,Product_ID
0,3,2013-01-01,1,0.0,3
1,5,2013-01-01,1,0.0,5
2,7,2013-01-01,1,0.0,7


In [33]:
products.head(3)

,Product_ID,Product_Code,SubCategory,Category,LeadTime,Price
0,20,HG001,Gardening Tools,Household Goods,2,50
1,27,HH001,Home Appliances,Household Goods,2,150
2,16,HK001,Kitchen,Household Goods,2,23


In [34]:
stores.head(3)

,Store_ID,City,State,Store_Type
0,1,Saint Paul,Minnesota,4
1,2,Saint Paul,Minnesota,4
2,3,Saint Paul,Minnesota,4


In [ ]:
##########################
sales2=pd.merge(sales,products.loc[:,['Product_ID','Price']],how="inner",on='Product_ID')

1) sales에 금액(Amt) 변수를 추가하시오.
* Amt = Qty * Price

In [52]:
# sales에는 Qty, Products에는 Price
# 두 df는 Product_ID 라는 속성을 포함하고 있음
sales2=pd.merge(sales,products,how="inner",on='Product_ID')
sales2.head()

,Order_ID,Date,Store_ID,Qty,Product_ID,Product_Code,SubCategory,Category,LeadTime,Price
0,3,2013-01-01,1,0.0,3,DB001,Beverage,Drink,2,8
1,36,2013-01-01,10,0.0,3,DB001,Beverage,Drink,2,8
2,69,2013-01-01,11,0.0,3,DB001,Beverage,Drink,2,8
3,102,2013-01-01,12,0.0,3,DB001,Beverage,Drink,2,8
4,135,2013-01-01,13,0.0,3,DB001,Beverage,Drink,2,8


In [53]:
sales2['Amt'] = sales2['Qty']*sales2['Price']
sales2.tail()

,Order_ID,Date,Store_ID,Qty,Product_ID,Product_Code,SubCategory,Category,LeadTime,Price,Amt
25105,55109,2013-01-31,54,3.000,32,GS001,Seafood,Grocery,3,34,102.000
25106,55142,2013-01-31,6,20.089,32,GS001,Seafood,Grocery,3,34,683.026
25107,55175,2013-01-31,7,43.647,32,GS001,Seafood,Grocery,3,34,1483.998
25108,55208,2013-01-31,8,42.621,32,GS001,Seafood,Grocery,3,34,1449.114
25109,55241,2013-01-31,9,24.986,32,GS001,Seafood,Grocery,3,34,849.524


2) 상품(Product_Code)별 매출액

In [54]:
sales2.groupby('Product_Code',as_index=False)['Amt'].sum()

,Product_Code,Amt
0,DA001,1.062816e+06
1,DB001,1.336522e+07
2,FB001,3.298673e+06
3,FF001,3.423657e+06
4,FM001,2.094948e+06
5,FP001,2.456785e+06
6,GA001,2.419344e+07
7,GE001,1.017210e+06
8,GM001,9.063631e+06
9,GS001,1.188430e+06


3) City별 카테고리리별 매출액

In [62]:
#1. sales2와 stores를 merge
stores2 = pd.merge(sales2,stores,how='inner',on='Store_ID')
#display(stores2)
#2 groupby
stores2.groupby(['City','Category'],as_index=False)['Amt'].sum()

,City,Category,Amt
0,Albany,Drink,2.367080e+05
1,Albany,Food,6.998561e+04
2,Albany,Grocery,3.676481e+05
3,Albany,Household Goods,2.875500e+05
4,Annapolis,Drink,3.507200e+05
...,...,...,...
79,Saint Paul,Household Goods,1.081224e+07
80,Wilmington,Drink,0.000000e+00
81,Wilmington,Food,0.000000e+00
82,Wilmington,Grocery,0.000000e+00


In [ ]:
### 번외 지역별로 어떤 카테고리의 매출액이 높은지?
# df.groupby(['지역','Category'],as_index=False)['금액'].sum()

4) 매출액이 가장 높은 주(state) top 3
- state별 매출액 합계
- 매출액 기준 정렬(내림차순)
- 정렬 후 head(3)하면 됨

In [73]:
temp = stores2.groupby('State',as_index=False)['Amt'].sum()
temp

,State,Amt
0,Arizona,4.192750e+06
1,California,8.744844e+05
2,Georgia,3.374833e+06
3,Indiana,2.410887e+06
4,Maryland,1.926923e+06
5,Massachusetts,9.879414e+05
6,Minnesota,4.733903e+07
7,New York,9.618918e+05
8,North Carolina,1.133845e+07
9,Ohio,1.650586e+06


In [74]:
temp.sort_values('Amt',ascending=False).head(3)

,State,Amt
6,Minnesota,4.733903e+07
8,North Carolina,1.133845e+07
0,Arizona,4.192750e+06


- 추가문제
#### 2013년 1월 셋째주 각 도시별 상품(SubCategory)별 매출액 비교
###### 1. 2013년 1월 셋째주
###### - `sales['Date']: pd.to_datetime()`으로 변환
###### - 주차 추출 : `sales['Week']` 만들기: `.dt.weekofyear` / `.dt.week`
###### - `sales['Week'] == 3` 인 것 가져오기. `sales.loc[__]`
###### 2. 각 도시별 상품(SubCategory)별
###### - 1의 결과, stores merge. (stores['City])
###### - 위 결과, products merge. (products['Price'],products['SubCategory'])
###### 3. 각 도시 별 상품(SubCategory)별 매출액 비교
###### - Amt 만들기 : Qty * Price
###### - groupby

- 수도코드/의사코드: 우리말로 알고리즘을 순차적으로 정리해 놓은 것

In [25]:
#1
sales['Date'] = pd.to_datetime(sales['Date'])
sales['Week'] = sales['Date'].dt.isocalendar().week
sales2 = sales.loc[sales['Week']==3]

In [34]:
#2
temp1 = pd.merge(sales2,stores)
temp2 = pd.merge(temp1,products)

In [38]:
#3
temp2['Amt'] = temp2['Qty']*temp2['Price']
temp2.groupby(['City','SubCategory'], as_index=False)['Amt'].sum()

,City,SubCategory,Amt
0,Albany,Agricultural products,67224.0
1,Albany,Alcohol,6336.0
2,Albany,BAKERY,7530.0
3,Albany,Beverage,45360.0
4,Albany,Cleaning,61365.0
...,...,...,...
289,Wilmington,Kitchen,0.0
290,Wilmington,Meats,0.0
291,Wilmington,Milk,0.0
292,Wilmington,Prepared,0.0
